In [1]:
import pandas as pd
import numpy as np
import datetime
import os
IRI_Selected_18=['LOS ANGELES','DALLAS/FT WORTH','NEW YORK','HOUSTON','WEST TEX/NEW MEXICO','PHOENIX/TUCSON','SOUTH CAROLINA',
                'BIRMINGHAM/MONTGOMERY','MISSISSIPPI','BALTIMORE/WASHINGTON','NEW ORLEANS/MOBILE',
                 'CHICAGO','ATLANTA','DETROIT','MIAMI/FT LAUDERDALE','HARRISBURG/SCRANTON','BOSTON',"ROANOKE"] #'RALEIGH/GREENSBORO','DENVER'

os.getcwd()

'/home/jian/SunnyD/Analysis_2019/IRI_DMA_mapping_Pctg'

In [2]:
IRI_Counties_both=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Sheet1',dtype=str)
IRI_Counties_IRI=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='IRI',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Nielsen',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen.columns=[['County','cleaned_name']]
IRI_Counties_both=IRI_Counties_both[(IRI_Counties_both['IRI_Label']!='nan') & (IRI_Counties_both['Nielsen_Label']!='nan')]

IRI_Counties_both=IRI_Counties_both[['County','County']]
IRI_Counties_both.columns=['County','cleaned_name']

Nielsen_counties=IRI_Counties_both.append(IRI_Counties_Nielsen).drop_duplicates().reset_index()
IRI_counties=IRI_Counties_both.append(IRI_Counties_IRI).drop_duplicates().reset_index()

del Nielsen_counties['index']
del IRI_counties['index']


In [3]:
All_DMA_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
All_DMA_zips=All_DMA_zips.iloc[:,[0,2,6,9]]
All_DMA_zips.columns=['zip_code','DMA','State','County']
All_DMA_zips['County']=All_DMA_zips['County']+"|"+All_DMA_zips['State']
All_DMA_zips['County']=All_DMA_zips['County'].apply(lambda x: x.replace(" ","").lower())
All_DMA_zips=All_DMA_zips[~All_DMA_zips['State'].isin(['AK','HI'])]
# 206 DMA after reomoving AK and HI

In [4]:
All_IRI_counties=pd.read_excel("/home/jian/SunnyD/Otherinput/Total_US_counties_IRI_markets_and_regions_cr_12272016.xlsx",dtype=str,sheetname='U.S. Counties',skiprows=1)
All_IRI_counties=All_IRI_counties[['County Name','State','IRI Market']]
All_IRI_counties['County']=All_IRI_counties['County Name']+"|"+All_IRI_counties['State']
All_IRI_counties['County']=All_IRI_counties['County'].apply(lambda x: x.replace(" ","").lower())
All_IRI_counties=pd.merge(All_IRI_counties,IRI_counties,on="County",how="left")
All_IRI_counties=All_IRI_counties[['IRI Market','cleaned_name']]

In [5]:
print(All_DMA_zips.shape)
All_DMA_zips=pd.merge(All_DMA_zips,Nielsen_counties,on="County",how="left")
All_DMA_zips=All_DMA_zips[['zip_code','DMA','County','cleaned_name']].rename(columns={"County":"Nielsen_County"})
print(All_DMA_zips.shape)

# Use only zip and DMA
All_DMA_zips.head(2)

(51066, 4)
(51066, 4)


,zip_code,DMA,Nielsen_County,cleaned_name
0,00501,NEW YORK,suffolk|ny,suffolk|ny
1,00544,NEW YORK,suffolk|ny,suffolk|ny


In [6]:
county_zip=All_DMA_zips[['zip_code','cleaned_name']].drop_duplicates()
All_IRI_zip=pd.merge(All_IRI_counties,county_zip,on="cleaned_name",how="left")
All_IRI_zip.head(2)

,IRI Market,cleaned_name,zip_code
0,BIRMINGHAM/MONTGOMERY,autauga|al,36003
1,BIRMINGHAM/MONTGOMERY,autauga|al,36006


In [7]:
IRI_zip_list=All_IRI_zip.groupby("IRI Market")['zip_code'].apply(list).to_frame().reset_index()
IRI_zip_list['zip_code']=IRI_zip_list['zip_code'].apply(set)

In [8]:
DMA_zip_list=All_DMA_zips.groupby("DMA")['zip_code'].apply(list).to_frame().reset_index()
DMA_zip_list['zip_code']=DMA_zip_list['zip_code'].apply(set)

In [9]:
# hold IRI
col_1=["IRI_Market","iri_zip_counts","DMA_count_overlapped","Covred_other_DMA_pctg","Be_covered_by_DMA"]
col_2=["IRI_Market","DMA","IRI_zip_count","DMA_zip_count","overlap_zip_count","overlap_pctg_of_DMA","covered_by_DMA"]
df_iri_with_DMAs_by_IRI=pd.DataFrame(columns=col_1)
df_iri_with_DMAs_by_IRI_and_DMA=pd.DataFrame(columns=col_2)
for ind_iri, row_iri in IRI_zip_list.iterrows():
    iri_name=row_iri['IRI Market']
    IRI_zip_set=row_iri['zip_code']
    dict_covered_other_dma=dict()
    dict_covered_by_dma=dict()
    
    for ind_dma,row_dma in DMA_zip_list.iterrows():
        dma_name=row_dma['DMA']
        dma_zip_set=row_dma['zip_code']
        intersection=IRI_zip_set.intersection(dma_zip_set)
        
        if len(intersection)>0:
            iri_covered_other_dma=np.round(len(intersection)/len(dma_zip_set),8)
            iri_covered_by_dma=np.round(len(intersection)/len(IRI_zip_set),8)
            dict_covered_other_dma.update({dma_name:iri_covered_other_dma})
            dict_covered_by_dma.update({dma_name:iri_covered_by_dma})
            
            df=pd.DataFrame({"IRI_Market":iri_name,
                             "DMA":dma_name,
                             "IRI_zip_count":len(IRI_zip_set),
                             "DMA_zip_count":len(dma_zip_set),
                             "overlap_zip_count":len(intersection),
                             "overlap_pctg_of_DMA":iri_covered_other_dma,
                             "covered_by_DMA":iri_covered_by_dma},index=[0])
            df_iri_with_DMAs_by_IRI_and_DMA=df_iri_with_DMAs_by_IRI_and_DMA.append(df)
    df_iri_with_DMAs_by_IRI=df_iri_with_DMAs_by_IRI.append(pd.DataFrame({"IRI_Market":iri_name,
                                                                         "iri_zip_counts":len(IRI_zip_set),
                                                                         "DMA_count_overlapped":len(dict_covered_other_dma),
                                                                         "Covred_other_DMA_pctg":str(dict_covered_other_dma),
                                                                         "Be_covered_by_DMA":str(dict_covered_by_dma)},
                                                                        index=[0]))
df_iri_with_DMAs_by_IRI=df_iri_with_DMAs_by_IRI[col_1]
df_iri_with_DMAs_by_IRI_and_DMA=df_iri_with_DMAs_by_IRI_and_DMA[col_2]


In [10]:
writer=pd.ExcelWriter("./SunnyD_mapping_overlap_hodling_IRI_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_iri_with_DMAs_by_IRI.to_excel(writer,"by_IRI",index=False)
df_iri_with_DMAs_by_IRI_and_DMA.to_excel(writer,"by_IRI_and_DMA",index=False)
writer.save()

In [11]:
# hold DMA
col_1=["DMA","DMA_zip_counts","IRI_count_overlapped","Covred_other_IRI_pctg","Be_covered_by_IRI"]
col_2=["DMA","IRI_Market","DMA_zip_count","IRI_zip_count","overlap_zip_count","overlap_pctg_of_IRI","covered_by_IRI"]
df_DMA_with_IRIs_by_DMA=pd.DataFrame(columns=col_1)
df_DMA_with_IRI_by_DMA_and_IRI=pd.DataFrame(columns=col_2)
for ind_dma,row_dma  in DMA_zip_list.iterrows():
    dma_name=row_dma['DMA']
    dma_zip_set=row_dma['zip_code']
    dict_covered_other_iri=dict()
    dict_covered_by_iri=dict()
    
    for ind_iri,row_iri in IRI_zip_list.iterrows():
        iri_name=row_iri['IRI Market']
        iri_zip_set=row_iri['zip_code']
        intersection=dma_zip_set.intersection(iri_zip_set)
        if len(intersection)>0:
            dma_covered_other_iri=np.round(len(intersection)/len(IRI_zip_set),8)
            dma_covered_by_iri=np.round(len(intersection)/len(dma_zip_set),8)
            dict_covered_other_iri.update({iri_name:dma_covered_other_iri})
            dict_covered_by_iri.update({iri_name:dma_covered_by_iri})
            
            df=pd.DataFrame({"DMA":dma_name,
                             "IRI_Market":iri_name,
                             "DMA_zip_count":len(dma_zip_set),
                             "IRI_zip_count":len(iri_zip_set),
                             "overlap_zip_count":len(intersection),
                             "overlap_pctg_of_IRI":dma_covered_other_iri,
                             "covered_by_IRI":dma_covered_by_iri},index=[0])
            df_DMA_with_IRI_by_DMA_and_IRI=df_DMA_with_IRI_by_DMA_and_IRI.append(df)
    df_DMA_with_IRIs_by_DMA=df_DMA_with_IRIs_by_DMA.append(pd.DataFrame({"DMA":dma_name,
                                                                         "DMA_zip_counts":len(dma_zip_set),
                                                                         "IRI_count_overlapped":len(dict_covered_other_iri),
                                                                         "Covred_other_IRI_pctg":str(dict_covered_other_iri),
                                                                         "Be_covered_by_IRI":str(dict_covered_by_iri)},
                                                                        index=[0]))
df_DMA_with_IRIs_by_DMA=df_DMA_with_IRIs_by_DMA[col_1]
df_DMA_with_IRI_by_DMA_and_IRI=df_DMA_with_IRI_by_DMA_and_IRI[col_2]

In [30]:
writer=pd.ExcelWriter("./SunnyD_mapping_overlap_hodling_DMA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_DMA_with_IRIs_by_DMA.to_excel(writer,"by_DMA",index=False)
df_DMA_with_IRI_by_DMA_and_IRI.to_excel(writer,"by_DMA_and_IRI",index=False)
writer.save()

# Format for deck

In [31]:
def format_pctg(x):
    y=""
    x=eval(x)
    for k,v in x.items():
        y=y+k+" ("+str(int(np.round(v*100,0)))+"%"+")"+", "
    
    return y[:len(y)-2]
    

In [32]:
df_iri_with_DMAs_by_IRI['formated_Covred_other_DMA_pctg']=df_iri_with_DMAs_by_IRI['Covred_other_DMA_pctg'].apply(lambda x: format_pctg(x))

In [33]:
writer=pd.ExcelWriter("./SunnyD_mapping_overlap_hodling_IRI_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_iri_with_DMAs_by_IRI.to_excel(writer,"by_IRI",index=False)
df_iri_with_DMAs_by_IRI_and_DMA.to_excel(writer,"by_IRI_and_DMA",index=False)
writer.save()